In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [ ]:
### 최종 모듈 수정 완료 ### 하ㅣㅈ디ㅏㄷ;ㅣ쟈넣댆;
years = os.listdir('/content/drive/MyDrive/동아리_스터디_대외활동/기상청 공모전/2-2날씨빅데이터 콘테스트')
areas = os.listdir('/content/drive/MyDrive/동아리_스터디_대외활동/기상청 공모전/2-2날씨빅데이터 콘테스트/{}'.format(years[0]))


def only_max_min_file(file_sort):
  global year_df, big_area_df_list
  year_df = []
  # 모든 파일 불러내기
  for year in years:
    big_area_df_list = []
    for area in areas:
      area_df_list = []
      path = '/content/drive/MyDrive/동아리_스터디_대외활동/기상청 공모전/2-2날씨빅데이터 콘테스트/{}/{}'.format(year, area)
      dir_list = os.listdir(path)
      print(dir_list)

      # 2012년 강원도 지역 파일 불러오기
      # dir_list = os.listdir('/content/drive/MyDrive/동아리_스터디_대외활동/기상청 공모전/2-2날씨빅데이터 콘테스트/{}/{}'.format(years[0], areas[0]))
      # dir_list

      # 지역 개수 구하기
      list_count = []
      for str_dir in dir_list:
        list_count.append(str_dir.split('_')[0])
      print(len(set(list_count)))

      # 일최저기온, 일최고기온 불러오고, 개수 구하기
      temp = []
      for str_dir in dir_list:
        if (str_dir.find(file_sort) != -1):
          temp.append(str_dir)
          print(str_dir)
      print(len(temp))

      if len(set(list_count)) == len(temp):
        print('파일 리스트 가져오기 성공')
        for csv_file in temp:
          csv_path = path + '/' + csv_file
          if file_sort == '일최저기온':
            area_df_list.append(open_file(csv_path, 'min')) # 지역 안의 여러 작은 지역 df 모으기 ------------------------1번 집계
          elif file_sort == '일최고기온':
            area_df_list.append(open_file(csv_path, 'max')) # 지역 안의 여러 작은 지역 df 모으기   


      else:
        print('파일 리스트 가져오기 실패')
        print('/content/drive/MyDrive/동아리_스터디_대외활동/기상청 공모전/2-2날씨빅데이터 콘테스트/{}/{}'.format(year, area))
        break

      # 지역 하나의 데이터의 평균 구하기
      # type 0: mean
      # type 1: min
      # type 2: max
      big_area_df_list.append(mean_area(area_df_list, 2)) # 강원, 경기 등의 df 담기 : 각각의 원소는 강원, 충북, 경기 등 ---------------------2번 집계-----------------------
    year_df.append(big_area_df_list) # year[0]=2012년, year[1]=2013년

def open_file(csv_path, aggfunc):
  # path와 csv파일 이름 붙이기
  df = pd.read_csv(csv_path)
  if df.empty:
    print('csv파일 읽기 실패')
    return
  df.head()
  df_need = play(df)
  df_date = make_df(df_need, aggfunc) # 한 csv 파일을 일자별로 mean, min, max 집계함  ------------------------------------1번 집계------------------------------------------
  return df_date

def play(df):
  # 달이 바뀌는 인덱스 추출하기
  index_start = df[df.iloc[:,0].str.startswith(' Start')].index
  date1 = df.columns[3].strip()[-8:]
  date = [date1]

  # 열 이름 바꾸기
  df = df.rename(columns = {df.columns[0] : 'day', df.columns[3] : 'value'})
  for i in index_start:
    date.append(df.iloc[i, 0].strip()[-8:])

  # date 열 추가
  df.loc[0:index_start[0]-1,'date'] = date[0]
  for i in range(len(date)-2):
    df.loc[index_start[i]:index_start[i+1]-1,'date'] = date[i+1]
  df.loc[index_start[len(date)-2]:, 'date'] = date[len(date)-1]


  

  # forecast가 4인 부분만 가져오기
  df_need = df[(df['forecast'] == 4) | (df['forecast'] == '+4')]
  df_need.reset_index(drop=True)
  df_need.iloc[:,0] = df_need.iloc[:,0].astype('int')

  # value 부분 float로 설정하기
  df_need['value'] = df_need['value'].astype('float')
  return df_need

  # 합친 테이블 만들기
def make_df(df_need, aggfunc): ################################################################################################################
  date_df = pd.pivot_table(df_need, index=[df_need.columns[-1], df_need.columns[0]], values=df_need.columns[3], aggfunc=aggfunc)
  print(date_df.head(10))
  return date_df

# # 날짜로 타입 변경하기
# def set_datetime(date_df):



def save_to_csv(date_df, file_name):
  # rename = file_name.split('.')[0] + 'edit.csv'
  # date_df.to_csv(rename)
  # print(rename+'저장완료')
  print(file_name + '저장완료')


# 지역 하나의 데이터의 평균 구하기
# type 0: mean
# type 1: min
# type 2: max
def mean_area(df_list, type):
  df_con = df_list[0]
  for i in range(1, len(df_list)):
    df_con = pd.concat([df_con, df_list[i]], 1)
  if type == 0:  
    df_con_mean = df_con.mean(axis=1).to_frame('max')
    print(df_con_mean)
    return df_con_mean
  elif type == 1:  
    df_con_mean = df_con.min(axis=1).to_frame('min')
    print(df_con_mean)
    return df_con_mean
  elif type == 2:  
    df_con_mean = df_con.max(axis=1).to_frame('max')
    print(df_con_mean)
    return df_con_mean

  

In [ ]:
# '일최저기온'
# '일최고기온'
# min = only_max_min_file('일최저기온')
max = only_max_min_file('일최고기온')

['송정동_파고_201601_201612.csv', '명륜1동_풍속_201601_201612.csv', '대관령면_강수형태_201601_201612.csv', '송정동_일최저기온_201601_201612.csv', '갈말읍_풍속_201601_201612.csv', '갈말읍_일최저기온_201601_201612.csv', '갈말읍_강수형태_201601_201612.csv', '갈말읍_일최고기온_201601_201612.csv', '대관령면_파고_201601_201612.csv', '대관령면_6시간적설_201601_201612.csv', '명륜1동_강수형태_201601_201612.csv', '갈말읍_6시간강수량_201601_201612.csv', '갈말읍_강수확률_201601_201612.csv', '갈말읍_습도_201601_201612.csv', '갈말읍_파고_201601_201612.csv', '갈말읍_6시간적설_201601_201612.csv', '갈말읍_3시간기온_201601_201612.csv', '중앙동_하늘상태_201601_201612.csv', '정선읍_강수형태_201601_201612.csv', '대관령면_강수확률_201601_201612.csv', '신사우동_강수형태_201601_201612.csv', '홍천읍_풍속_201601_201612.csv', '중앙동_파고_201601_201612.csv', '중앙동_일최저기온_201601_201612.csv', 'ᄌ

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

              value
date     day       
20160101 1      6.9
         2      8.1
         3      7.8
         4      5.9
         5      3.1
         6      2.9
         7      2.2
         8      2.2
         9      3.2
         10     3.1
              value
date     day       
20160101 1      7.0
         2      8.0
         3      8.0
         4      6.0
         5      3.0
         6      3.0
         7      2.0
         8      2.0
         9      3.0
         10     3.0
              value
date     day       
20160101 1      8.0
         2      9.0
         3      9.0
         4      8.0
         5      4.0
         6      4.0
         7      2.0
         8      3.0
         9      4.0
         10     4.0
              value
date     day       
20160101 1     12.0
         2     12.0
         3     12.0
         4      9.0
         5      7.0
         6      8.0
         7      6.0
         8      6.0
         9      8.0
         10     8.0
              value
date     day       


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

              value
date     day       
20160101 1      7.0
         2      8.0
         3      8.0
         4      6.0
         5      3.0
         6      2.0
         7      2.0
         8      2.0
         9      3.0
         10     3.0
              value
date     day       
20160101 1     11.0
         2     11.0
         3     11.0
         4      9.0
         5      7.0
         6      7.0
         7      5.0
         8      5.0
         9      6.0
         10     5.0
              value
date     day       
20160101 1      7.0
         2      8.0
         3      8.0
         4      7.0
         5      3.0
         6      3.0
         7      2.0
         8      2.0
         9      3.0
         10     4.0
              value
date     day       
20160101 1      6.7
         2      8.8
         3      8.5
         4      8.7
         5      3.3
         6      3.3
         7      2.1
         8      2.0
         9      4.1
         10     4.0
              value
date     day       


...

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


              value
date     day       
20120101 1      1.0
         2      0.0
         3      3.0
         4     -0.9
         5      1.0
         6      2.0
         7      2.0
         8      3.0
         9      3.0
         10     2.0


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


              value
date     day       
20120101 1      1.0
         2      1.0
         3     -1.9
         4      0.0
         5      2.0
         6      2.0
         7      3.0
         8      3.0
         9      3.0
         10     2.0
              value
date     day       
20120101 1      1.0
         2      1.0
         3     -0.9
         4      0.0
         5      2.0
         6      2.0
         7      2.0
         8      3.0
         9      2.0
         10     2.0
              max
date     day     
20120101 1    1.1
         2    1.0
         3    3.0
         4    0.0
         5    2.1
...           ...
20121201 27   0.0
         28   3.0
         29   3.0
         30  -5.0
         31  -4.0

[366 rows x 1 columns]


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:123: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only


In [ ]:
# '일최저기온'
# '일최고기온'
min = only_max_min_file('일최저기온')
# max = only_max_min_file('일최고기온')

['갈말읍_12시간강수량_201301_201312.csv', '갈말읍_3시간기온_201301_201312.csv', '갈말읍_강수확률_201301_201312.csv', '갈말읍_일최고기온_201301_201312.csv', '갈말읍_일최저기온_201301_201312.csv', '갈말읍_파고_201301_201312.csv', '갈말읍_풍속_201301_201312.csv', '갈말읍_하늘상태_201301_201312.csv', '대관령면_6시간강수량_201301_201312.csv', '갈말읍_12시간신적설_201301_201312.csv', '대관령면_6시간적설_201301_201312.csv', '갈말읍_6시간강수량_201301_201312.csv', '대관령면_강수형태_201301_201312.csv', '갈말읍_6시간적설_201301_201312.csv', '갈말읍_강수형태_201301_201312.csv', '대관령면_습도_201301_201312.csv', '갈말읍_습도_201301_201312.csv', '대관령면_풍향_201301_201312.csv', '갈말읍_풍향_201301_201312.csv', '대관령면_12시간강수량_201301_201312.csv', '명륜1동_12시간강수량_201301_201312.csv', '대관령면_12시간신적설_201301_201312.csv', '명륜1동_12시간신적설_201301_201312.

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[selected_item_labels] = value
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

              value
date     day       
20130101 1     -6.8
         2    -11.8
         3     -4.1
         4     -4.1
         5     -2.0
         6     -1.2
         7     -1.3
         8     -2.1
         9     -6.9
         10    -3.0
              value
date     day       
20130101 1      3.0
         2     -2.9
         3      2.0
         4      4.0
         5      4.0
         6      4.0
         7      5.0
         8      6.0
         9      2.0
         10     4.0
              value
date     day       
20130101 1     -2.9
         2     -6.9
         3     -3.9
         4     -0.9
         5      0.0
         6      0.0
         7      1.0
         8      1.0
         9     -2.9
         10     1.0
              value
date     day       
20130101 1      4.0
         2     -0.9
         3      4.0
         4      5.0
         5      5.0
         6      5.0
         7      6.0
         8      6.0
         9      3.0
         10     5.0


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[selected_item_labels] = value
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

              value
date     day       
20130101 1     -4.8
         2     -7.8
         3     -5.7
         4     -3.5
         5     -1.5
         6     -1.4
         7     -0.3
         8     -0.5
         9     -3.6
         10     1.1
              value
date     day       
20130101 1     -4.9
         2     -7.9
         3     -5.9
         4     -3.9
         5     -1.9
         6     -1.9
         7     -0.9
         8     -0.9
         9     -3.9
         10     1.0
              value
date     day       
20130101 1     -4.2
         2     -6.1
         3     -3.1
         4     -2.2
         5     -0.1
         6      0.8
         7      1.8
         8      0.8
         9     -3.1
         10     1.9
              value
date     day       
20130101 1     -4.9
         2     -6.9
         3     -4.9
         4     -2.9
         5     -0.9
         6     -0.9
         7      1.0
         8      0.0
         9     -2.9
         10     1.0


...

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


              value
date     day       
20120101 1      2.2
         2      1.2
         3      1.1
         4      0.2
         5      2.2
         6      3.1
         7      3.2
         8      3.2
         9      3.1
         10     2.1


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

              value
date     day       
20120101 1      2.0
         2      2.0
         3      1.0
         4      0.0
         5      2.0
         6      3.0
         7      3.0
         8      3.0
         9      3.0
         10     2.0
              value
date     day       
20120101 1      1.0
         2      1.0
         3      0.0
         4     -1.9
         5      1.0
         6      1.0
         7      1.0
         8      2.0
         9      2.0
         10     2.0
              value
date     day       
20120101 1      2.0
         2      1.0
         3      1.0
         4      0.0
         5      2.0
         6      3.0
         7      3.0
         8      3.0
         9      3.0
         10     2.0
              max
date     day     
20120101 1    2.2
         2    2.0
         3    1.1
         4    0.2
         5    2.2
...           ...
20121201 27   0.0
         28  -1.0
         29   4.0
         30  -4.0
         31  -4.0

[366 rows x 1 columns]


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:123: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only


### real

In [ ]:
len(year_df)

5

In [ ]:
len(year_df[0])

10

In [ ]:
len(year_df[1])

13

In [ ]:
years

['2016', '2013', '2014', '2015', '2012']

In [ ]:
########### 최종 데이터 출력
final_mean_list = []
for area_index in range(len(year_df[1])):
  aaa = pd.concat([year_df[0][area_index], year_df[1][area_index], year_df[2][area_index], year_df[3][area_index], year_df[4][area_index]], 0)
  final_mean_list.append(aaa)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  after removing the cwd from sys.path.


IndexError: ignored

In [ ]:
len(final_mean_list)

- 최고, 최저 기온 따로 돌리기 *2 수정하기
- 최고, 최적 기온은 각각 max, min으로 함수 설정
- 두 열 합치고, 지역 이름까지 집어넣기


In [ ]:
final_mean_list[:][0]

In [ ]:
final_mean_list[4]

In [ ]:
areas
final_mean_list[0]['area'] = areas[0]
# 지역 열 추가하기
for i in range(1, len(final_mean_list)):
  each_df = final_mean_list[i]
  each_df['area'] = areas[i]

In [ ]:
# axis 행으로 합치기
df_con = final_mean_list[0]
for i in range(1, len(final_mean_list)):
  df_con = pd.concat([df_con, final_mean_list[i]], 0)
print(df_con)

In [ ]:
df_con.head(10)

In [ ]:
df_max_max = df_con.copy()
df_max_max

In [ ]:
df_min_min.columns = ['min_min', 'area']
# df_min_mean.columns = ['min_mean', 'area']
# df_max_mean.columns = ['max_mean', 'area']
df_max_max.columns = ['max_max', 'area']

In [ ]:
df_min_min

min_min    area
date     day                   
20130101 1   -21.900000  강원
         2   -23.900000  강원
         3   -23.900000  강원
         4   -22.900000  강원
         5   -21.799999  강원
...                 ...     ...
20121201 27   -5.000000  충북
         28   -7.000000  충북
         29  -11.000000  충북
         30  -18.000000  충북
         31  -17.000000  충북

[24607 rows x 2 columns]

In [ ]:
df_max_max

max_max    area
date     day                 
20130101 1        4.0  강원
         2       -0.9  강원
         3        4.0  강원
         4        5.0  강원
         5        5.0  강원
...               ...     ...
20121201 27       0.0  충북
         28      -1.0  충북
         29       4.0  충북
         30      -4.0  충북
         31      -4.0  충북

[24606 rows x 2 columns]

In [ ]:
need_to_concat = [df_min_min, df_max_max],

In [ ]:
# 인덱스 초기화, 재설정
for i in [df_min_min, df_max_max]:
  # i.reset_index(inplace=True) 위에거 실행하고 이거 주석처리하고 밑에거 다시 실행
  i.set_index(['date','day', 'area'], inplace=True)

In [ ]:
df_final = pd.concat([df_min_min,df_max_max], 1)
df_final

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  """Entry point for launching an IPython kernel.


min_min  max_max
date     day area                    
20120101 1   강원    -13.9      6.0
             경기     -12.2      1.1
             경남     -8.9      8.0
             경북    -12.2      7.0
             광주      -3.9      4.0
...                      ...      ...
20151201 31  전남     -4.8     10.0
             전북     -8.0      8.0
             제주        3.5     12.5
             충남     -7.0      8.0
             충북     -8.0      6.7

[24607 rows x 2 columns]

In [ ]:
df_final.to_csv('0731_data.csv')

In [ ]:
df_min_min.to_csv('aa.csv',encoding="utf-8-sig")

In [ ]:
pd.read_csv('0731_data.csv')

,date,day,area,min_min,max_max
0,20120101,1,강원,-13.9,6.0
1,20120101,1,경기,-12.2,1.1
2,20120101,1,경남,-8.9,8.0
3,20120101,1,경북,-12.2,7.0
4,20120101,1,광주,-3.9,4.0
...,...,...,...,...,...
24602,20151201,31,전남,-4.8,10.0
24603,20151201,31,전북,-8.0,8.0
24604,20151201,31,제주,3.5,12.5
24605,20151201,31,충남,-7.0,8.0
